Moving Average Crossover Strategy

In [62]:
import numpy as np
import pandas as pd
import yfinance as yf

def _stock_prices_dataset(ticker):
  
  #import data from yahoo finance
  stock_prices_dataset = yf.download(ticker, start="2020-01-01", end="2021-01-01")
  
  stock_prices_dataset = stock_prices_dataset[["Adj Close"]].reset_index(drop=True)
  
  stock_prices_dataset["returns"] = np.log(stock_prices_dataset["Adj Close"]).diff()
  
  stock_prices_dataset["returns"] = stock_prices_dataset["returns"].shift(-1)

  return stock_prices_dataset

In [63]:
print(_stock_prices_dataset("AAPL"))

[*********************100%%**********************]  1 of 1 completed

      Adj Close   returns
0     73.059433 -0.009770
1     72.349129  0.007937
2     72.925621 -0.004714
3     72.582664  0.015958
4     73.750244  0.021018
..          ...       ...
248  129.514435  0.035141
249  134.146652 -0.013404
250  132.360504 -0.008563
251  131.231903 -0.007732
252  130.221100       NaN

[253 rows x 2 columns]


In [5]:
def _sma_stock_prices_dataset(stock_prices_dataset, long_sma = 200, short_sma = 50):

  stock_prices_dataset[f"{long_sma}_day_sma"] = stock_prices_dataset["adj_close"].rolling(long_sma).mean()
  stock_prices_dataset[f"{short_sma}_day_sma"] = stock_prices_dataset["adj_close"].rolling(short_sma).mean()
  stock_prices_dataset = stock_prices_dataset.dropna()
  stock_prices_dataset = stock_prices_dataset.reset_index(drop=True)

  return stock_prices_dataset

In [6]:
def _sma_crossover_dataset(sma_stock_prices_dataset, long_sma = 200, short_sma = 50):

  sma_crossover_datalist = []

  sma_stock_prices_dataset["crossover_signal"] = np.where(sma_stock_prices_dataset[f"{short_sma}_day_sma"] >= sma_stock_prices_dataset[f"{long_sma}_day_sma"], 1, 0)
  sma_stock_prices_dataset["prev_crossover_signal"] = sma_stock_prices_dataset["crossover_signal"].shift(1, fill_value=0)

  sma_stock_prices_dataset["buy"] = (sma_stock_prices_dataset["crossover_signal"] == 1) & (sma_stock_prices_dataset["prev_crossover_signal"] == 0)
  sma_stock_prices_dataset["buy"]  = sma_stock_prices_dataset["buy"] .replace({True: 1, False: 0})
  sma_stock_prices_dataset["sell"] = (sma_stock_prices_dataset["crossover_signal"] == 0) & (sma_stock_prices_dataset["prev_crossover_signal"] == 1)
  sma_stock_prices_dataset["sell"]  = sma_stock_prices_dataset["sell"] .replace({True: 1, False: 0})
  
  holding = False
  for sma_stock_prices in sma_stock_prices_dataset.to_dict("records"):
    
    if holding and sma_stock_prices.get("sell") == 1:
      holding = False
    
    if not holding and sma_stock_prices.get("buy") == 1:
      holding = True

    sma_stock_prices["hold"] = holding
    sma_crossover_datalist.append(sma_stock_prices)
  
  sma_crossover_dataset = pd.DataFrame(sma_crossover_datalist)

  return sma_crossover_dataset

In [7]:
def _sma_returns_data(sma_crossover_dataset):

  sma_crossover_dataset["algo_returns"] = sma_crossover_dataset["returns"] * sma_crossover_dataset["hold"]
  sma_return = round(sma_crossover_dataset["algo_returns"].sum() * 100, 3)
  buy_and_hold_return = round(sma_crossover_dataset["returns"].sum() * 100, 3)
  total_entries = sma_crossover_dataset["buy"].sum()

  return sma_return, buy_and_hold_return, total_entries

In [22]:
def sma_handler(ticker, long_sma = 200, short_sma = 50):

  stock_prices_dataset = _stock_prices_dataset(ticker)
  # start_date = stock_prices_dataset.iloc[0]['date']
  # sma_stock_prices_dataset = _sma_stock_prices_dataset(stock_prices_dataset, long_sma, short_sma)
  # sma_crossover_dataset = _sma_crossover_dataset(sma_stock_prices_dataset, long_sma, short_sma)
  # sma_return, buy_and_hold_return, total_entries = _sma_returns_data(sma_crossover_dataset)
  
  # print(f"Start Date: {start_date}")
  # print(f"SMA Return: {sma_return}")
  # print(f"Buy and Hold Return: {buy_and_hold_return}")
  # print(f"Total Entries: {total_entries}")

sma_handler("AAPL")


[*********************100%%**********************]  1 of 1 completed

KeyError: "None of [Index(['date', 'adj close'], dtype='object')] are in the [columns]"